In [7]:
# coding: utf-8

# In[120]:

import jieba
from jieba import posseg as pseg
from collections import Counter, defaultdict
import random


# In[165]:

def load_file(file_name):
    f = open(file_name, 'r')
    lines = f.readlines()
    f.close()
    segs = []
    for line in lines:
        line = line.strip().decode('utf-8')
        words = pseg.cut(line)
        words = [key for (key, flag) in words]
        for word in words:
            segs.append(word)            
    return segs


# In[166]:

def create_model(segs, n, padding):
    Delimiter = [u'。',u'！',u'？']
    lm = defaultdict(Counter)
    if n < 1:
        n = 1
    post_segs = []
    for i in range(n):
        post_segs.append(padding)
    for word in segs:
        post_segs.append(word)
        context = tuple(post_segs[-n-1:-1])
        lm[context][word] += 1
        if word in Delimiter:
            for j in range(n):
                post_segs.append(padding)  
                
    for key, cnt in lm.items():
        s = float(sum(cnt.values()))
        for word in cnt:
            cnt[word] /= s
    return lm
    


# In[167]:

def generate_head(n, pad):
    head = []
    for i in range(n):
       head.append(pad)
    return head


# In[168]:

def generate_word(lm_counter, context):
    r = random.random()
    s = 0.0
    for word, value in lm_counter[context].items():
        s += value
        if s > r:
            return word


# In[169]:

def generate_sentence(lm, start):    
    context = start
    sentence = []
    text = ''

    while (True):
        word = generate_word(lm, context)
        if word == None: 
            break
        else:
            text += word
            temp = list(context)[1:]
            temp.append(word)
            context = tuple(temp) 
            
    sentence.append(text.encode('utf-8'))
    return sentence


# In[170]:

def generate_sample_text(lm, n, sentence_count, padding):
    heads = generate_head(n, padding)
    start = tuple(heads)
    text = ''
    for i in range(sentence_count):
        sentences = generate_sentence(lm, start)
        sentences.append('\r\n')
        text += ''.join(sentences)
    return text


# In[171]:

def demo_model_from_file(file_name, max_n):
    segs = load_file(file_name)
    padding = u'%'
    
    if max_n > 5:
        max_n = 5
    else:
        if max_n < 1:
            max_n = 1
            
    for n in range(1,max_n):
        print '-----------ngram = %d-----------' % (n)
        lm = create_model(segs, n, padding)
        text = generate_sample_text(lm, n, 10, padding)
        print text
        print '--------------------------------'

In [8]:
demo_model_from_file('../AssisantEvaluate/西游记.txt', 5)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.316 seconds.
Prefix dict has been built succesfully.


-----------ngram = 1-----------
拜我把行李，十分狼狈，倒树头儿报仇！
”众僧耳朵，三藏道：“响震山川社稷袄，算了一哄了唐僧携手出昆仑顶上，崎岖，《起信论经，拜，未曾定是也。
”行者笑道：“二哥的们口里哼哼道：“兄弟合意运。
及参拜佛求经》一部一百一十六狮顿首道：单用着唐僧得命感毗蓝毡包袱之内，决不肯供饭，争奈你便也罢，双晦气！
’字，恐师父说不了，你只管阿绵花屎来！
这殿下，黄芦叶斗。
”那白森森的火罩儿，后妃跌跌与他回国不题。
”罗刹再拜，仔细观看，两只手，流星还不曾见我们斗智哩。
霜雪，孤掌难鸣。
这面前无净坛使，抬头见一座高山，翠柏丛中排善士。

--------------------------------
-----------ngram = 2-----------
那怪王急抽身走入里面，即问当驾官打扫御花园馆阁楼亭，且待我成功之后，不换，不斋丑的，我本是陈清家一秤金？
”玉帝道：“大哥莫忙，莫忙！
”猴王道：“大王放心，只顾使棒支吾？
过了吊桥，径奔朝门而去。
两个前进，把那三间禅堂，再来说话！
大圣在先看时，朕即立以三事，倒也祥光隐隐，不见了马匹，那个脱命的女菩萨，弟子拿不动；待要开言骂战。
三个魔头与三个徒弟貌丑，甚不通理。
也罢，怎么哄他出来，叫我们不是？
”行者跪在面前道：“拿来，行者漫门缝儿里不是？
我这里有的是鞭简瓜锤，刀枪钺斧，教我女儿乖滑，他就捞了几下，看看菩萨的金紧禁三个箍儿咒》，不得留情，一时间疼痛难禁，这些和尚缘路来，吆喝沙僧：“长兄勿得胡谈，任教出入乘龙。

--------------------------------
-----------ngram = 3-----------
”就去叫抬出轿来。
”诸天道：“我也正要些儿吃哩。
”沙僧闻言道：“二哥胡缠！
”原来那怪九个头，转转都是眼睛，看得真实，恐怕他来此害你，因见气数还旺，不敢下手，见他们与太子这样绸缪，怎不黄昏！
行者上前迎住道：“哥哥，没甚手段，拿不住那妖精，取衣西进也。
”菩萨道：“师父，拿住一个对头在此，老孙独力扶持，有何不可！
若是大圣打杀了走兽飞禽，呜的一声都哄然而去了。
闻知尊府敬僧，故此拜见，求一斋就行。
你老子今日也斋僧，还是小可。
话表唐僧师徒四众，师父去倒换关文，特来朝参千岁。

---------------